# Importing the libraries

In [1]:
#Importing the required libraries
import pandas as pd
import numpy as np
from collections import Counter as c
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
!pip install ibm_watson_machine_learning

# Loading the dataset

In [5]:
dataset = pd.read_csv('credit_train.csv')
dataset.head()


FileNotFoundError: [Errno 2] No such file or directory: 'credit_train.csv'

In [ ]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_7e6ae28e13b9413f84d4dfd46572ec7d = 'https://s3.eu.cloud-object-storage.appdomain.cloud'
else:
    endpoint_7e6ae28e13b9413f84d4dfd46572ec7d = 'https://s3.private.eu.cloud-object-storage.appdomain.cloud'

client_7e6ae28e13b9413f84d4dfd46572ec7d = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='bxlj9AG3rsG1Qz7n5BmbUOr_rdpRindmfNjGWlhvZ5pu',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_7e6ae28e13b9413f84d4dfd46572ec7d)

body = client_7e6ae28e13b9413f84d4dfd46572ec7d.get_object(Bucket='loanstatusprediction-donotdelete-pr-r6nersecsn4kus',Key='credit_train.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

dataset = pd.read_csv(body)
dataset.head()


In [ ]:
#finding the number of rows and columns
dataset.shape

In [ ]:
#lists out the names of the columns 
dataset.columns   

In [ ]:
# It will display the first five rows of the dataset
dataset.head()    

# Null Values

In [ ]:
#lists the  null values in every column of the dataset
dataset.isnull().any()

In [ ]:
#Finding the  sum of null values in every column of the dataset
dataset.isnull().sum()

# Categorical Columns

In [ ]:
#lists the columns with categorical data
object_train_df=dataset.select_dtypes(include=['object'])    
object_train_df.columns

# Numerical Columns

In [ ]:
#lists the columns with numerical data
num_train_df=dataset.select_dtypes(include=['int','float'])     
num_train_df.columns

# Dropping Loan Status Null Values and Labeling it

In [ ]:
dataset.dropna(subset=['Loan Status'], inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le =LabelEncoder()
dataset['Loan Status'] = le.fit_transform(dataset['Loan Status'])

In [ ]:
dataset

# Term column Labeling

In [ ]:
dataset['Term'].replace(('Short Term','Long Term'),(0,1),inplace=True)
dataset.head()

# Scaling Credit Score Column

In [ ]:
#Applying lamda function
dataset['Credit Score'] = dataset['Credit Score'].apply(lambda val: (val /10) if val>850 else val)

# Handling Null values of Credit Score Column

In [ ]:
do_nothing = lambda: None
cscoredf = dataset[dataset['Term']==0]
stermAVG = cscoredf['Credit Score'].mean()
lscoredf = dataset[dataset['Term']==1]
ltermAVG = lscoredf['Credit Score'].mean()
dataset.loc[(dataset.Term==0) & (dataset['Credit Score'].isnull()),'Credit Score'] = stermAVG
dataset.loc[(dataset.Term==1) & (dataset['Credit Score'].isnull()),'Credit Score'] = ltermAVG

In [ ]:
dataset['Credit Score'] = dataset['Credit Score'].apply(lambda val: "Poor" if np.isreal(val) and val < 580 else val)
dataset['Credit Score'] = dataset['Credit Score'].apply(lambda val: "Average" if np.isreal(val) and (val >= 580 and val < 670) else val)
dataset['Credit Score'] = dataset['Credit Score'].apply(lambda val: "Good" if np.isreal(val) and (val >= 670 and val < 740) else val)
dataset['Credit Score'] = dataset['Credit Score'].apply(lambda val: "Very Good" if np.isreal(val) and (val >= 740 and val < 800) else val)
dataset['Credit Score'] = dataset['Credit Score'].apply(lambda val: "Exceptional" if np.isreal(val) and (val >= 800 and val <= 850) else val)

# Analying the data

In [ ]:
## The graph lists out the counts in an ascending way
dataset['Credit Score'].value_counts().sort_values(ascending = True).plot(kind='bar', title = 'Credit Score category')

# Annual income column



In [ ]:
# Prints the sum of missing values in the column Annual Income.
print("There are",dataset['Annual Income'].isna().sum(), "Missing Annual Income Values.")

In [ ]:
#by using fillna function we are filling the null values with the mean method inplace where it is true.
dataset['Annual Income'].fillna(dataset['Annual Income'].mean(), inplace=True)

In [ ]:
#finding the data shape
dataset.shape

In [ ]:
#By using the counter function we are to get the count of Good, Very Good and Average.
from collections import Counter as c
print(c(dataset['Credit Score']))  #returns the class count values 

In [ ]:
##applying label encoder
dataset['Credit Score'] = le.fit_transform(dataset['Credit Score'])
c(dataset['Credit Score'])

# Home Ownership Column

In [ ]:
#Home Ownership Column   we are sorting the elements with values in ascending order. 
dataset['Home Ownership'].value_counts().sort_values(ascending = True).plot(kind='bar', title="Number of Loan based on Home ownership")

In [ ]:
print(c(dataset['Home Ownership']))
dataset['Home Ownership'] =  le.fit_transform(dataset['Home Ownership'])
print(c(dataset['Home Ownership']))

# Years in current job

In [ ]:
dataset['Years in current job']=dataset['Years in current job'].str.extract(r"(\d+)")
dataset['Years in current job'] = dataset['Years in current job'].astype(float)

In [ ]:
expmean = dataset['Years in current job'].mean()

In [ ]:
dataset['Years in current job'].fillna(expmean, inplace=True)
dataset['Years in current job'].fillna(expmean, inplace=True)

In [ ]:
dataset

# Dropping unwanted columns

In [ ]:
dataset = dataset.drop(['Loan ID','Customer ID','Purpose'], axis=1)

# Credit Problems

In [ ]:
#Normalizing 
dataset['Credit Problems'] = dataset['Number of Credit Problems'].apply(lambda x: "No Credit Problem" if x==0 else ("Some Credit promblem" if x>0 and x<5 else "Major Credit Problems"))

In [ ]:
print(c(dataset['Credit Problems']))
dataset['Credit Problems'] = le.fit_transform(dataset['Credit Problems'])
print(c(dataset['Credit Problems']))

# Credit Age


In [ ]:
dataset['Credit Age'] = dataset['Years of Credit History'].apply(lambda x: "Short Credit Age" if x<5 
                                else ("Good Credit Age" if x>5 and x<17 else "Exceptional Credit Age"))

In [ ]:
print(c(dataset['Credit Age']))
dataset['Credit Age'] = le.fit_transform(dataset['Credit Age'])
print(c(dataset['Credit Age']))

In [ ]:
dataset = dataset.drop(['Months since last delinquent','Number of Open Accounts',
                  'Maximum Open Credit','Current Credit Balance','Monthly Debt'],axis=1)

# Tax Liens

In [ ]:
dataset['Tax Liens'] = dataset['Tax Liens'].apply(lambda x: "No Tax Lien" if x==0 else ("Some Tax Liens" if x>0 and x<3 else "Many Tax Liens"))

In [ ]:
print(c(dataset['Tax Liens']))
dataset['Tax Liens'] = le.fit_transform(dataset['Tax Liens'])
print(c(dataset['Tax Liens']))

# Bankruptcies

In [ ]:
dataset['Bankruptcies'] = dataset['Bankruptcies'].apply(lambda x: "No bankruptcies" if x==0  else ("Some Bankruptcies" if x>0 and x<3 else "Many Bankruptcies"))

In [ ]:
print(c(dataset['Bankruptcies']))
dataset['Bankruptcies'] = le.fit_transform(dataset['Bankruptcies'])
print(c(dataset['Bankruptcies']))

# Annual Income

In [ ]:
meanxoutlier = dataset[dataset['Annual Income'] < 99999999.00 ]['Annual Income'].mean()
stddevxoutlier = dataset[dataset['Annual Income'] < 99999999.00 ]['Annual Income'].std()
poorline = meanxoutlier -  stddevxoutlier
richline = meanxoutlier + stddevxoutlier

In [ ]:
dataset['Annual Income'] = dataset['Annual Income'].apply(lambda x: "Low Income" if x<=poorline  else ("Average Income" if x>poorline and x<richline else "High Income"))

In [ ]:
print(c(dataset['Annual Income']))
dataset['Annual Income'] = le.fit_transform(dataset['Annual Income'])
print(c(dataset['Annual Income']))

# Current Loan Amount

In [ ]:
lmeanxoutlier = dataset[dataset['Current Loan Amount'] < 99999999.00 ]['Current Loan Amount'].mean()
lstddevxoutlier = dataset[dataset['Current Loan Amount'] < 99999999.00 ]['Current Loan Amount'].std()
lowrange = lmeanxoutlier - lstddevxoutlier
highrange = lmeanxoutlier + lstddevxoutlier
print(lowrange, highrange)

In [ ]:
dataset['Current Loan Amount'] = dataset['Current Loan Amount'].apply(lambda x: "Small Loan" if x<=lowrange else ("Medium Loan" if x>lowrange and x<highrange else "Big Loan"))

In [ ]:
print(c(dataset['Current Loan Amount']))
dataset['Current Loan Amount'] = le.fit_transform(dataset['Current Loan Amount'])
print(c(dataset['Current Loan Amount']))

In [ ]:
dataset.shape

# Seperating Dependent and Independent Columns

In [ ]:
y = dataset['Loan Status']
X = dataset.drop(['Loan Status'],axis=1)

In [ ]:
dataset.head()

# Performing Train and test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
#By using DecisionTree we are fitting the model
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)


In [ ]:
X_train.shape

In [ ]:
y_pred_dt =dt.predict(X_test)  #prediction
c(y_pred_dt)

In [ ]:
X_train[0]

In [ ]:
X_test

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_pred_dt,y_test)

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
                 "url": "https://eu-gb.ml.cloud.ibm.com",
                 "apikey":"bxlj9AG3rsG1Qz7n5BmbUOr_rdpRindmfNjGWlhvZ5pu"
                  }
client =  APIClient(wml_credentials)

In [ ]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources']if item['entity']["name"] == space_name)['metadata']['id'])

In [ ]:
space_uid= guid_from_space_name(client, 'newspace')
print("Space UID=" +space_uid)

In [ ]:
client.set.default_space(space_uid)

In [ ]:
client.software_specifications.list()

In [ ]:
software_spec_uid = client.software_specifications.get_uid_by_name("default_py3.7")
software_spec_uid

In [ ]:
model_details = client.repository.store_model(model=dt,meta_props={
 client.repository.ModelMetaNames.NAME:"Loan_Prediction",
 client.repository.ModelMetaNames.TYPE:"scikit-learn_0.23", 
 client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid }
                                        )
model_id = client.repository.get_model_uid(model_details)


In [ ]:
model_id

In [ ]:
dataset


# Creating a pickle file dumping the model in it

In [ ]:
 #importing the pickle file
import pickle 
#Dumping the model into the pickle file
pickle.dump(dt,open('loan.pkl','wb'))   